## Grouping and Pivoting in DataFrames

Classifying our data is one of the most important tasks ofa Data Scientist, and Pandas gives this functionality to us on a silver plate. We use what we call, __Multi-level Indexing__ or __Hierarchical Indexing__ to Group and Pivot our data. First let us import the necessary libraries.

In [2]:
import numpy as np
import pandas as pd


Now we are ready. Let us learn more about it. 

### Hierarchical Indexing

If we want, we can create DataFrames with multiple indices. The combinations of these indices would be used for our indexing. Such indexing is called __Hierarchical or Multi-Level Indexing__. 

You can create multiple indices by passing a list of column locations in the `index_col` keyword argument. It will follow the following syntax - 

<b><code>pd.read_csv(data,index_col=[y<sub>1</sub>,y<sub>2</sub>...y<sub>n</sub>])</code></b>

Let us see an example.

In [ ]:
df = pd.read_csv('https://media-doselect.s3.amazonaws.com/generic/NMgEjwkAEGGQZBoNYGr9Ld7w0/rating.csv',
                 index_col=[1,2])
df.head()

,,ID,Rating
Department,Office,,
Finance,New Delhi,U2F26,3.4
Marketing,New Delhi,U2M61,3.9
Sales,New Delhi,U1S15,2.8
HR,Mumbai,U1H87,2.1
Sales,New Delhi,U1S51,4.6


In [ ]:
df.index

MultiIndex([(  'Finance', 'New Delhi'),
            ('Marketing', 'New Delhi'),
            (    'Sales', 'New Delhi'),
            (       'HR',    'Mumbai'),
            (    'Sales', 'New Delhi'),
            (       'HR', 'New Delhi'),
            ('Marketing', 'New Delhi'),
            ('Marketing', 'Bangalore'),
            (    'Sales', 'Bangalore'),
            (       'HR', 'Bangalore'),
            ...
            (  'Finance', 'New Delhi'),
            ('Marketing', 'New Delhi'),
            (    'Sales', 'New Delhi'),
            (    'Sales',    'Mumbai'),
            (    'Sales', 'Bangalore'),
            (    'Sales', 'New Delhi'),
            ('Marketing', 'Bangalore'),
            (  'Finance', 'Bangalore'),
            (    'Sales', 'Bangalore'),
            (    'Sales', 'New Delhi')],
           names=['Department', 'Office'], length=533)

Here, you can see that we have our Multi-level indices here. To access any row, we need to mention both the indices.

In multi-level indexing, we use the following syntax to access the rows, and the columns will be accessed in the same way. Here, all the columns that are the indices won't be mentioned in the column part of the syntax:

__`loc[[(index1,index2,....)],[column1,column2....]]`__

Let us see an example.


In [ ]:
df.loc[[('Finance','New Delhi')]].head(5)

ID  Rating
Department Office                  
Finance    New Delhi  U2F26     3.4
           New Delhi  U2F25     4.4
           New Delhi  U3F41     4.9
           New Delhi  U2F89     4.4
           New Delhi  U3F87     3.4

In [ ]:
df.loc[[('Finance','New Delhi'),('Finance','Bangalore')],['ID']]

ID
Department Office          
Finance    New Delhi  U2F26
           New Delhi  U2F25
           New Delhi  U3F41
           New Delhi  U2F89
           New Delhi  U3F87
...                     ...
           Bangalore  U1F51
           Bangalore  U1F21
           Bangalore  U1F75
           Bangalore  U2F65
           Bangalore  U3F53

[91 rows x 1 columns]

See? This is how we created a DataFrame with multiple indices. Now, what if __we want to set multiple indices for an existing DataFrame__ ? For that, we use the __`set_index()`__ method, which takes a list of columns to be set as index, and the `inplace` keyword argument. It has the following syntax - 

__`DataFrame.set_index([list-of-columns],inplace=)`__

Let us see it in action.

In [ ]:
rate= pd.read_csv('https://media-doselect.s3.amazonaws.com/generic/NMgEjwkAEGGQZBoNYGr9Ld7w0/rating.csv')
rate.index

RangeIndex(start=0, stop=533, step=1)

In [ ]:
rate['Training']=rate['Rating'].apply(lambda x: 'Yes' if x<=3.5 else 'No')
rate.columns

Index(['ID', 'Department', 'Office', 'Rating', 'Training'], dtype='object')

In [ ]:
rate.set_index(['Department','Training'],inplace=True)
rate.head()

,,ID,Office,Rating
Department,Training,,,
Finance,Yes,U2F26,New Delhi,3.4
Marketing,No,U2M61,New Delhi,3.9
Sales,Yes,U1S15,New Delhi,2.8
HR,Yes,U1H87,Mumbai,2.1
Sales,No,U1S51,New Delhi,4.6


Now, let us say, we want to know which department is the most efficient. The department that has the least percentage of people that need to be trained is the one we are searching for. 

Hence, this is where we use multilevel indexing to the best effect. We will follow the following steps here - 

- We will first calculate the ratio of 'Yes' in Training in each department to the total no of people in that department. 
- A dictionary will be created, having the department names as index and the ratios as values.
- Then we will create a panda series from the dictionary. 
- Finally, we will use the `idxmin()` to return the name of the department. 

Let us see the code now.

In [ ]:
norm={i:rate.loc[[(i,'Yes')]]['Rating'].count()/rate.loc[[(i)]]['Rating'].count() for i in ['Finance','Marketing','HR','Sales']}
norm_ser=pd.Series(norm)
norm_ser.idxmin()

'HR'

This is how multilevel indexing is beneficial in data science. Let us now move towards its another use - Grouping and Pivoting.

### Grouping Data

Grouping and aggregation are some of the most frequently used operations in data analysis, especially while performing exploratory data analysis (EDA), where comparing summary statistics across groups of data is common.

Grouping analysis can be thought of as having three parts:
- Splitting the data into groups (e.g., groups of customer segments, product categories, etc.)
- Applying a function to each group (e.g., the mean or total sales of each customer segment)
- Combining the results into a data structure showing summary statistics.

To start this grouping analysis, we use an in-built method in Pandas – the __`groupby()`__ method. This method takes an __index name as a parameter__, and __groups the data by that index name__. 


There are somethings about this function that need to be considered – 
- It is usually followed by an aggregate function like `sum()`, `count()`, `mean()`, `prod()` etc. Without these functions, it would not return a DataFrame, but a pandas object. 
- It is better to perform this operation on selected columns, to get meaningful data.
- You need to have an index that can be grouped, for this function to work. If your function has a primary key index (all unique values), then it won’t work. 

It has the following syntax –

__`identifier[[column-list]].groupby(index-name).aggfunc()`__

Let us see an example. 


In [19]:
sales=pd.read_excel('https://github.com/yashj1301/Python3-UpGrad-UMich/blob/master/Python%203.x/Upgrad/Modules/Module%203%20-%20Python%20for%20Data%20Science/Session%203%20-%20Pandas/Data/sales.xlsx?raw=true')
sales.shape 

(23, 5)

In [20]:
sales.set_index(['Market'],inplace=True)
sales.head()

,Region,No_of_Orders,Profit,Sales
Market,,,,
Africa,Western Africa,251,-12901.51,78476.06
Africa,Southern Africa,85,11768.58,51319.50
Africa,North Africa,182,21643.08,86698.89
Africa,Eastern Africa,110,8013.04,44182.60
Africa,Central Africa,103,15606.30,61689.99


Here, our index is `Market`, which is a categorical variable, and hence, it can be grouped. Now, let us find out how many regions are there in each market. 

In [22]:
sales[['Region']].groupby('Market').count()

,Region
Market,
Africa,5
Asia Pacific,6
Europe,4
LATAM,3
USCA,5


Here, we can see the number of regions in each market. This is how our `groupby()` method works. Let us take another dataset to apply this method.

In [28]:
fire=pd.read_csv('https://github.com/yashj1301/Python3-UpGrad-UMich/raw/master/Python%203.x/Upgrad/Modules/Module%203%20-%20Python%20for%20Data%20Science/Session%203%20-%20Pandas/Data/forestfires.csv')
fire.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


Now, let us group this data by `month` and `day`, and __find out the mean values for wind and rain__ from there. First, let us set these two columns as our multiple indices. 

In [35]:
fire.set_index(['month','day'],inplace=True)
fire.head()

X  Y  FFMC   DMC     DC  ISI  temp  RH  wind  rain  area
month day                                                          
mar   fri  7  5  86.2  26.2   94.3  5.1   8.2  51   6.7   0.0   0.0
oct   tue  7  4  90.6  35.4  669.1  6.7  18.0  33   0.9   0.0   0.0
      sat  7  4  90.6  43.7  686.9  6.7  14.6  33   1.3   0.0   0.0
mar   fri  8  6  91.7  33.3   77.5  9.0   8.3  97   4.0   0.2   0.0
      sun  8  6  89.3  51.3  102.2  9.6  11.4  99   1.8   0.0   0.0

Now, it is time to find the mean of the columns. Let us see it in action.

In [41]:
fire[['wind','rain','area']].groupby(['month','day']).mean()

wind  rain       area
month day                           
apr   fri  3.100000   0.0   0.000000
      mon  3.100000   0.0   3.350000
      sat  4.500000   0.0   0.000000
      sun  5.666667   0.0  20.376667
      thu  5.800000   0.0   7.770000
...             ...   ...        ...
sep   sat  3.460000   0.0  61.804400
      sun  3.955556   0.0  14.010741
      thu  3.357143   0.0   5.356190
      tue  3.431579   0.0  26.352105
      wed  3.378571   0.0  13.028571

[64 rows x 3 columns]

This is how the `groupby()` method is used in Pandas DataFrames. Now, let us look at pivoting in DataFrames. 

### Pivoting Data

A pivot table is a handy tool to represent a DataFrame in a structured and simplified manner. It acts as an __alternative to the `groupby()`__ function in Pandas. Pivot tables __provide excel-like functionalities__ to create aggregate tables.

To create pivot tables, there is <font color="red">no necessary requirement to have multi-level indices</font>; In fact, index with no labels is also appropriate, because we define everything inside its method.

This method is the __`pivot_table()`__ method, which creates a DataFrame equivalent to a pivot table. It takes the following keyword arguments – 

- __`values`__ – This argument takes the list of column names on which the aggregate function needs to be performed. 
- __`index`__ – This argument defines the index (or list of indices) on the basis of which, grouping will be done. 
- __`aggfunc`__– This argument takes the name (or dictionary) of the aggregate function to be performed. In case different functions need to be performed on different columns, then dictionary is used.
- __`columns`__ (optional) – This argument is used when you want to create a crosstab between two categorical variables. This can also take a list of all the columns that you want to group by. 

This method <font color="darkred">does not overwrite the original DataFrame</font>. It returns a pivot table DataFrame. Hence, it needs to be stored in a variable. It follows the following syntax – 

__`identifier.pivot_table(values=[list-of-values], index=[list-of-indices], columns=[list-of-columns], aggfunc=)`__

Let us now see it in action. 


In [72]:
fire=pd.read_csv('https://github.com/yashj1301/Python3-UpGrad-UMich/raw/master/Python%203.x/Upgrad/Modules/Module%203%20-%20Python%20for%20Data%20Science/Session%203%20-%20Pandas/Data/forestfires.csv')
fire.head()

,X,Y,month,day,FFMC,DMC,DC,ISI,temp,RH,wind,rain,area
0,7,5,mar,fri,86.2,26.2,94.3,5.1,8.2,51,6.7,0.0,0.0
1,7,4,oct,tue,90.6,35.4,669.1,6.7,18.0,33,0.9,0.0,0.0
2,7,4,oct,sat,90.6,43.7,686.9,6.7,14.6,33,1.3,0.0,0.0
3,8,6,mar,fri,91.7,33.3,77.5,9.0,8.3,97,4.0,0.2,0.0
4,8,6,mar,sun,89.3,51.3,102.2,9.6,11.4,99,1.8,0.0,0.0


First, let us create a pivot table that calculates the mean of the columns `temp` and `wind`, and is __grouped by `month`__. Let us see.

In [88]:
piv=fire.pivot_table(values=['temp','wind'],index='month',aggfunc=np.mean)
piv

,temp,wind
month,,
apr,12.044444,4.666667
aug,21.631522,4.086413
dec,4.522222,7.644444
feb,9.635000,3.755000
jan,5.250000,2.000000
jul,22.109375,3.734375
jun,20.494118,4.135294
mar,13.083333,4.968519
may,14.650000,4.450000


We can see that the `columns` parameter wasn’t used. This is because we don’t need crosstabs right now. Next, let us create a pivot table having crosstabs between `day` and `month`, which will return the __maximum value of the column `wind`__. Let us see now.

In [101]:
cross=fire.pivot_table(index='month', columns='day',values='wind', aggfunc=np.max)
cross

day,fri,mon,sat,sun,thu,tue,wed
month,,,,,,,
apr,3.1,3.1,4.5,9.4,5.8,NaN,2.7
aug,8.9,5.4,8.0,7.6,6.7,7.6,5.4
dec,4.9,8.5,NaN,8.5,4.9,8.5,8.0
feb,9.4,6.3,4.0,4.0,3.1,6.3,3.1
jan,NaN,NaN,3.1,0.9,NaN,NaN,NaN
jul,8.0,4.9,5.4,6.3,4.9,5.4,7.6
jun,5.4,5.4,3.1,4.5,9.4,NaN,4.9
mar,9.4,5.8,8.5,6.3,5.8,7.6,8.0
may,4.0,NaN,4.9,NaN,NaN,NaN,NaN


This is how we use the `columns` argument to create crosstabs, and how we use the `pivot_table()` method in DataFrames.